In [93]:
import pandas as pd
import json
from sklearn.preprocessing import MinMaxScaler
import numpy as np

In [82]:
f = open("prices.json")
prices_json = json.load(f)

In [83]:
stocks = []
prices = pd.DataFrame()
for stock in prices_json:
    temp = pd.DataFrame(prices_json[stock])
    temp['stock'] = stock
    prices = pd.concat([prices, temp])
    

In [74]:
prices.dropna(inplace=True)
#prices = prices[~prices.index.duplicated(keep='first')]
prices.drop(columns=['volume'], inplace=True)

In [75]:
prices['time'] = pd.to_datetime(prices['time'])
prices.sort_values('time', inplace=True)
#prices.set_index('time', inplace=True)

In [76]:
cols_to_convert = [col for col in prices.columns if col not in ['stock', 'time']]
prices[cols_to_convert] = prices[cols_to_convert].astype(float)


In [77]:
cols_to_convert

['open', 'high', 'low', 'close']

In [78]:
scaler = MinMaxScaler()
prices[cols_to_convert] = scaler.fit_transform(prices[cols_to_convert])

In [87]:
prices = pd.get_dummies(prices, columns = ['stock'])

time    open    high     low   close  volume  stock_ALGM  \
0     20240202  09:30:00  173.77  178.86  173.05  174.67      -1       False   
1     20240202  11:00:00  174.67  177.62  174.14  177.39      -1       False   
2     20240202  13:00:00  177.39  179.00  176.86  178.48      -1       False   
3     20240202  15:00:00  178.48  178.73  177.45  177.66      -1       False   
4     20240205  09:30:00  179.05  180.00  171.11  172.85      -1       False   
...                  ...     ...     ...     ...     ...     ...         ...   
2987  20230202  09:30:00   84.23   88.04   83.62   87.48      -1       False   
2988  20230202  11:00:00   87.48   88.46   86.43   88.16      -1       False   
2989  20230202  13:00:00   88.16   88.94   86.36   86.36      -1       False   
2990  20230202  15:00:00   86.36   88.46   86.02   88.36      -1       False   
2991  20230203  09:30:00   86.66   88.15   86.06   87.55      -1       False   

      stock_AZPN  stock_PCOR  
0          False        True  
1          False        True  
2          False        True  
3          False        True  
4          False        True  
...          ...         ...  
2987        True       False  
2988        True       False  
2989        True       False  
2990        True       False  
2991        True       False  

[8976 rows x 9 columns]

In [114]:
seq_length = 250 #Time stamp for 3 months
X_list, y_list = [], []

for stock_name, group in prices.groupby('stock'):
    #display(group)
    data_array = group[cols_to_convert].values
    close_prices = group['close'].values

    for i in range(len(group) - seq_length):
        
        # A = group.iloc[:i + seq_length]
        # A['time'] = pd.to_datetime(A['time'])
        # print((A.iloc[-1]['time'] - A.iloc[0]['time']).days)
        # break

        X_seq = data_array[i : i + seq_length]  # shape: (seq_length, num_features)
        last_close = close_prices[i + seq_length - 1]
        next_close = close_prices[i + seq_length]  # the day after the window
        label = 1 if next_close > last_close else 0

        X_list.append(X_seq)
        y_list.append(label)

X = np.array(X_list)
y = np.array(y_list)

2742
5484
8226
